In [1]:
!git clone https://github.com/NVlabs/stylegan.git

Cloning into 'stylegan'...
remote: Enumerating objects: 83, done.
remote: Total 83 (delta 0), reused 0 (delta 0), pack-reused 83
Unpacking objects: 100% (83/83), done.


In [2]:
cd stylegan/

/home/jovyan/stylegan-experiments/stylegan


Download our preprocessed children's drawings

In [3]:
!wget http://roberttwomey.com/downloads/drawings_resized.tar.gz

--2020-03-07 20:11:14--  http://roberttwomey.com/downloads/drawings_resized.tar.gz
Resolving roberttwomey.com (roberttwomey.com)... 64.90.42.74
Connecting to roberttwomey.com (roberttwomey.com)|64.90.42.74|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 344737216 (329M) [application/gzip]
Saving to: ‘drawings_resized.tar.gz’

drawings_resized.ta 100%[===================>] 328.77M  83.0MB/s    in 5.2s    

2020-03-07 20:11:19 (63.5 MB/s) - ‘drawings_resized.tar.gz’ saved [344737216/344737216]



Extract training images from tarball:

In [4]:
!tar -xvf drawings_resized.tar.gz

drawings_resized/
drawings_resized/drawings_original_00000004.jpg_9c206c7b-83b9-41cf-8560-f600e3010685.JPEG
drawings_resized/drawings_original_00000005.jpg_e302a9ad-1aea-45bb-867b-a61abc387b2d.JPEG
drawings_resized/drawings_original_00000006.jpg_6d939ab4-ffe3-4e17-a573-c5137ce5fde6.JPEG
drawings_resized/drawings_original_00000007.jpg_4d89673e-295a-4168-a132-610afcfa7930.JPEG
drawings_resized/drawings_original_00000008.jpg_2cab0111-7ece-4ba6-b1e3-0e711943979d.JPEG
drawings_resized/drawings_original_00000009.jpg_296c489e-fd4f-4a39-b86b-b27b00910752.JPEG
drawings_resized/drawings_original_00000010.jpg_1e905b5e-f136-49b0-b115-49696549a242.JPEG
drawings_resized/drawings_original_00000011.jpg_66811b8d-6a21-4a67-a10f-ef7010f550de.JPEG
drawings_resized/drawings_original_00000012.jpg_3d471a89-4e16-43d7-89d1-72a98194f9fe.JPEG
drawings_resized/drawings_original_00000013.jpg_3022e9c9-d976-4a15-b62f-0cda54d0ec1e.JPEG
drawings_resized/drawings_original_00000014.jpg_da5fc433-1809-44a4-bcf1-0123529916

download latest model for transfer learning

In [5]:
!wget https://drive.google.com/uc?export=download&confirm=CFts&id=1cJQtMeTy_QldOP7n64F8stCDXY6Esup9

generate tfrecords

In [7]:
!python dataset_tool.py create_from_images datasets/smalls/ stylegan/drawings_resized/

Traceback (most recent call last):
  File "dataset_tool.py", line 21, in <module>
    import dnnlib.tflib as tflib
  File "/home/jovyan/stylegan-experiments/stylegan/dnnlib/tflib/__init__.py", line 8, in <module>
    from . import autosummary
  File "/home/jovyan/stylegan-experiments/stylegan/dnnlib/tflib/autosummary.py", line 31, in <module>
    from . import tfutil
  File "/home/jovyan/stylegan-experiments/stylegan/dnnlib/tflib/tfutil.py", line 34, in <module>
    def shape_to_list(shape: Iterable[tf.Dimension]) -> List[Union[int, None]]:
AttributeError: module 'tensorflow' has no attribute 'Dimension'
